In [1]:
import numpy as np
import pandas as pd

In [67]:
dataset = [{1}, {2}, {3}, {4}, {5}, {8}, {9}, {10}, {11}, {12}, {24}, {28}, {32}, {36}, {40}]

In [69]:
while len(dataset) > 4:
    cluster(dataset)

[{1, 2}, {3}, {4}, {5}, {8}, {9}, {10}, {11}, {12}, {24}, {28}, {32}, {36}, {40}]
[{1, 2}, {3, 4}, {5}, {8}, {9}, {10}, {11}, {12}, {24}, {28}, {32}, {36}, {40}]
[{1, 2}, {3, 4}, {5}, {8, 9}, {10}, {11}, {12}, {24}, {28}, {32}, {36}, {40}]
[{1, 2}, {3, 4}, {5}, {8, 9}, {10, 11}, {12}, {24}, {28}, {32}, {36}, {40}]
[{1, 2}, {3, 4, 5}, {8, 9}, {10, 11}, {12}, {24}, {28}, {32}, {36}, {40}]
[{1, 2}, {3, 4, 5}, {8, 9}, {10, 11, 12}, {24}, {28}, {32}, {36}, {40}]
[{1, 2, 3, 4, 5}, {8, 9}, {10, 11, 12}, {24}, {28}, {32}, {36}, {40}]
[{1, 2, 3, 4, 5}, {8, 9, 10, 11, 12}, {24}, {28}, {32}, {36}, {40}]
[{1, 2, 3, 4, 5, 8, 9, 10, 11, 12}, {24}, {28}, {32}, {36}, {40}]
[{1, 2, 3, 4, 5, 8, 9, 10, 11, 12}, {24, 28}, {32}, {36}, {40}]
[{1, 2, 3, 4, 5, 8, 9, 10, 11, 12}, {24, 28}, {32, 36}, {40}]


In [68]:
def cluster(dataset):
    min_dist = 999999
    clusters_to_combine = (0, 0)
    min_card = 99999
    idx = -1
    
    for i, cluster in enumerate(dataset):
        if i == len(dataset) - 1:
            break
        dist = abs(max(dataset[i]) - min(dataset[i+1]))
        card = len(dataset[i]) + len(dataset[i+1])
        if dist < min_dist:
            clusters_to_combine = dataset[i], dataset[i+1]
            min_dist = dist
            min_card = card
            idx = i
        elif dist == min_dist and card < min_card:
            clusters_to_combine = dataset[i], dataset[i+1]
            min_dist = dist
            min_card = card
            idx = i
    
#     print("Joining the clusters: ", clusters_to_combine)
#     print("single link distance:", min_dist)
#     print("Total cardinality:", min_card)
    joined_cluster = clusters_to_combine[0].union(clusters_to_combine[1])
    dataset[idx] = joined_cluster
    dataset.pop(idx+1)
    print(dataset)

In [43]:
cluster(dataset)

Joining the clusters:  ({32}, {36})
single link distance: 4
Total cardinality: 2
Resulting cluseters:
[{1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12}, {24, 28}, {32, 36}, {40}]


In [17]:
abs(max({1, 2}) - min({3}))

1

In [58]:
dataset = [{1}, {2}, {3}, {4}, {5}, {8}, {9}, {10}, {11}, {12}, {24}, {28}, {32}, {36}, {40}]

In [59]:
k = 3

In [60]:
initial_cents = [1, 8, 12]

In [61]:
clusters = [set(), set(), set()]

In [81]:
def closest_centroid(centroids, item):
    min_dist = 999999999
    idx = -1
    for j in range(3):
        cent = centroids[j]
        dist = abs(item - cent)
#         print("distance between {0} and {1}: {2}".format(el, cent, dist))
        if dist < min_dist:
            min_dist = dist
            idx = j
    return idx

In [62]:
for x in dataset:
    el = next(iter(x))
    min_dist = 999999999
    idx = -1
    for j in range(3):
        cent = initial_cents[j]
        dist = abs(el - cent)
#         print("distance between {0} and {1}: {2}".format(el, cent, dist))
        if dist < min_dist:
            min_dist = dist
            idx = j
    # Add to cluster j
#     print("Assigning {0} to cluster {1}".format(el, idx))
    clusters[idx] = clusters[idx].union(x)

In [63]:
clusters

[{1, 2, 3, 4}, {5, 8, 9, 10}, {11, 12, 24, 28, 32, 36, 40}]

In [71]:
cents = [1, 8, 12]

In [73]:
for j in range(3):
    cents[j] = sum(clusters[j]) / len(clusters[j])
print(cents)

[2.5, 8.0, 26.142857142857142]


In [96]:
new_clusters = [set(), set(), set()]
for cluster in clusters:
    for item in cluster:
        idx = closest_centroid(cents, item)
        new_clusters[idx] = new_clusters[idx].union({item})
print(new_clusters)

[{1, 2, 3, 4, 5}, {8, 9, 10, 11, 12}, {32, 36, 24, 40, 28}]


In [95]:
clusters = new_clusters
for j in range(3):
    cents[j] = sum(clusters[j]) / len(clusters[j])
print(cents)

[3.0, 10.0, 32.0]


In [97]:
df = pd.read_csv('prof_data.csv')

In [98]:
df

,Department,Rank,Age,Salary
0,CS,Full,old,high
1,Math,Associate,young,low
2,CS,Assistant,young,low
3,Math,Full,old,high
4,Physics,Associate,young,medium
5,Physics,Assistant,young,low
6,Math,Full,old,medium
7,CS,Associate,young,high
8,EE,Full,old,high
9,EE,Associate,young,medium


In [100]:
df.Salary.value_counts(normalize=True)

high      0.4
low       0.3
medium    0.3
Name: Salary, dtype: float64

In [113]:
salary_probs = df.groupby('Salary').size().div(len(df))

In [116]:
df.groupby(['Age', 'Salary']).size().div(len(df)).div(salary_probs, axis=0, level='Salary')

Age    Salary
old    high      0.750000
       medium    0.333333
young  high      0.250000
       low       1.000000
       medium    0.666667
dtype: float64

In [117]:
0.3333 * 0.3333 * 0.3333 * 0.3

0.011107778111099998

In [118]:
transactions = [{'A', 'B', 'C', 'D', 'H'}, {'A', 'B', 'C', 'E'}, {'A', 'B','E','F','H'}, {'A','C','H'}, {'B','D','H'}]

In [119]:
min_supp = 0.4
min_conf = 0.5

In [130]:
freq_itemsets = []

In [138]:
def compute_freq(item, transactions):
    count = 0
    for trans in transactions:
        if item.issubset(trans):
            count += 1
    return count / len(transactions)

In [132]:
for item in ['A', 'B', 'C','D', 'E','F', 'H']:
    freq = compute_freq(item, transactions)
    if freq >= min_supp:
        freq_itemsets.append(({item}, freq))

In [133]:
freq_itemsets

[({'A'}, 0.8),
 ({'B'}, 0.8),
 ({'C'}, 0.6),
 ({'D'}, 0.4),
 ({'E'}, 0.4),
 ({'H'}, 0.8)]

In [134]:
import itertools

In [147]:
freq_itemsets_2 = []

In [148]:
for pair in itertools.product(freq_itemsets, repeat=2):
    if pair[0] == pair[1]:
        continue
    itemset = pair[0][0].union(pair[1][0])
    freq = compute_freq(itemset, transactions)
    if freq >= min_supp:
        freq_itemset = (itemset, freq)
        if freq_itemset in freq_itemsets_2:
            continue
        freq_itemsets_2.append(freq_itemset)
    

In [150]:
freq_itemsets_3 = []

In [151]:
for pair in itertools.product(freq_itemsets_2, repeat=2):
    if pair[0] == pair[1]:
        continue
    itemset = pair[0][0].union(pair[1][0])
    if len(itemset) > 3 or len(itemset) < 3:
        continue
    
    freq = compute_freq(itemset, transactions)
    if freq >= min_supp:
        freq_itemset = (itemset, freq)
        if freq_itemset in freq_itemsets_3:
            continue
        freq_itemsets_3.append(freq_itemset)
    

In [152]:
freq_itemsets_3

[({'A', 'B', 'C'}, 0.4),
 ({'A', 'B', 'E'}, 0.4),
 ({'A', 'B', 'H'}, 0.4),
 ({'A', 'C', 'H'}, 0.4),
 ({'B', 'D', 'H'}, 0.4)]

In [153]:
freq_itemsets_4 = []

In [154]:
for pair in itertools.product(freq_itemsets_3, repeat=2):
    if pair[0] == pair[1]:
        continue
    itemset = pair[0][0].union(pair[1][0])
    if len(itemset) > 4 or len(itemset) < 4:
        continue
    
    freq = compute_freq(itemset, transactions)
    if freq >= min_supp:
        freq_itemset = (itemset, freq)
        if freq_itemset in freq_itemsets_3:
            continue
        freq_itemsets_4.append(freq_itemset)
    

In [188]:
closed_itemsets


[]

In [189]:
supp_40 = [x[0] for x in freq_itemsets_3 if x[1] == 0.4 ] + \
    [x[0] for x in freq_itemsets_2 if x[1] == 0.4 ] + \
    [x[0] for x in freq_itemsets if x[1] == 0.4 ]
supp_60 = [x[0] for x in freq_itemsets_3 if x[1] == 0.6 ] + \
    [x[0] for x in freq_itemsets_2 if x[1] == 0.6 ] + \
    [x[0] for x in freq_itemsets if x[1] == 0.6 ]
supp_80 = [x[0] for x in freq_itemsets_3 if x[1] == 0.8 ] + \
    [x[0] for x in freq_itemsets_2 if x[1] == 0.8 ] + \
    [x[0] for x in freq_itemsets if x[1] == 0.8 ]

closed_itemsets = []

while len(supp_40) > 0:
    subset_added = supp_40[0]
    closed_itemsets.append(subset_added)
    # Remove subsets
    supp_40 = [x for x in supp_40 if not x.issubset(subset_added)]


In [190]:
supp_60

[{'A', 'B'}, {'A', 'C'}, {'A', 'H'}, {'B', 'H'}, {'C'}]

In [191]:
while len(supp_60) > 0:
    subset_added = supp_60[0]
    closed_itemsets.append(subset_added)
    # Remove subsets
    supp_60 = [x for x in supp_60 if not x.issubset(subset_added)]
    
while len(supp_80) > 0:
    subset_added = supp_80[0]
    closed_itemsets.append(subset_added)
    # Remove subsets
    supp_80 = [x for x in supp_80 if not x.issubset(subset_added)]

In [192]:
closed_itemsets

[{'A', 'B', 'C'},
 {'A', 'B', 'E'},
 {'A', 'B', 'H'},
 {'A', 'C', 'H'},
 {'B', 'D', 'H'},
 {'A', 'B'},
 {'A', 'C'},
 {'A', 'H'},
 {'B', 'H'},
 {'A'},
 {'B'},
 {'H'}]

In [194]:
maximal_itemsets = [{'A', 'B', 'C'},
 {'A', 'B', 'E'},
 {'A', 'B', 'H'},
 {'A', 'C', 'H'},
 {'B', 'D', 'H'}]

In [195]:
maximal_itemsets

[{'A', 'B', 'C'},
 {'A', 'B', 'E'},
 {'A', 'B', 'H'},
 {'A', 'C', 'H'},
 {'B', 'D', 'H'}]

In [200]:
def compute_count(itemset, transactions):
    count = 0
    for trans in transactions:
        if itemset.issubset(trans):
            count += 1
    return count

In [206]:
for itemset in maximal_itemsets:
    # try size 1
    ls_seen = []
    
    for item in itemset:
        ls = {item}
        rs = itemset - {item}
        conf = compute_count(itemset, transactions) / compute_count(ls, transactions)
        if conf >= min_conf:
            ls_seen.append(ls)
            print("Confidence for {0} -> {1}: {2}".format(ls, rs, conf))
    # try size 2
    
    for pair in itertools.product(itemset, repeat=2):
        if pair[0] == pair[1]:
            continue
        ls = {pair[0]}.union({pair[1]})
        rs = itemset - ls
        conf = compute_count(itemset, transactions) / compute_count(ls, transactions)
        if conf >= min_conf and ls not in ls_seen:
            ls_seen.append(ls)
            print("Confidence for {0} -> {1}: {2}".format(ls, rs, conf))
    print("=====")

Confidence for {'B'} -> {'C', 'A'}: 0.5
Confidence for {'A'} -> {'B', 'C'}: 0.5
Confidence for {'C'} -> {'B', 'A'}: 0.6666666666666666
Confidence for {'B', 'A'} -> {'C'}: 0.6666666666666666
Confidence for {'B', 'C'} -> {'A'}: 1.0
Confidence for {'C', 'A'} -> {'B'}: 0.6666666666666666
=====
Confidence for {'E'} -> {'B', 'A'}: 1.0
Confidence for {'B'} -> {'E', 'A'}: 0.5
Confidence for {'A'} -> {'B', 'E'}: 0.5
Confidence for {'B', 'E'} -> {'A'}: 1.0
Confidence for {'E', 'A'} -> {'B'}: 1.0
Confidence for {'B', 'A'} -> {'E'}: 0.6666666666666666
=====
Confidence for {'H'} -> {'B', 'A'}: 0.5
Confidence for {'B'} -> {'A', 'H'}: 0.5
Confidence for {'A'} -> {'B', 'H'}: 0.5
Confidence for {'B', 'H'} -> {'A'}: 0.6666666666666666
Confidence for {'A', 'H'} -> {'B'}: 0.6666666666666666
Confidence for {'B', 'A'} -> {'H'}: 0.6666666666666666
=====
Confidence for {'H'} -> {'C', 'A'}: 0.5
Confidence for {'C'} -> {'A', 'H'}: 0.6666666666666666
Confidence for {'A'} -> {'C', 'H'}: 0.5
Confidence for {'C', '

In [ ]:
%